In [1]:
from support_funcs import *
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go

In [2]:
df = get_yield_curves()

In [ ]:
df.head()

In [3]:
melted_df = pd.melt(df, id_vars=['date'], value_vars=['1_m', '3_m','6_m','1_y','2_y',
                                   '3_y','5_y','7_y','10_y','20_y','30_y'])
melted_df.columns = ['date','term','rate']

In [ ]:
melted_df.head()

## Scatter Plot

In [ ]:
data = [go.Scattergl(x=df['10_y'],y=df['30_y'],mode='lines+markers',
                     marker=dict(
                         size=12,
                         color='rgb(51,204,255)',
                         symbol='diamond-tall-open-dot',
                         line=dict(width=1)
                     ))]
layout = go.Layout(title='10 yr rate vs 30 yr rate', 
                   xaxis=dict(title='10 yr'),yaxis=dict(title='30 yr'), 
                   hovermode='closest')
fig = go.Figure(data=data,layout=layout)
pyo.plot(fig)

## Line Plot 

In [ ]:
def plot_tenor(tenor):
    df_temp = df[df[tenor].first_valid_index():]
    trace0 = go.Scattergl(x=df_temp['date'],
                         y=df_temp[tenor],mode='lines+markers',
                         name='lines',
                         line=dict(
                             color='rgb(0, 54, 135)',
                             width=1),
                         marker=dict(
                             color='rgb(0, 0, 0)',
                             opacity=0.1,
                             size=.5)
                         )
    
    data = [trace0]
    tenor,horizon = tenor.split('_')
    if horizon == 'y':
        label = tenor + ' year yield'
    else:
        label = tenor + ' month yield'
    layout = go.Layout(title=label, 
                       xaxis=dict(title='Date'),yaxis=dict(title=label), 
                       hovermode='closest')
    fig = go.Figure(data=data,layout=layout)
    pyo.plot(fig)

In [ ]:
plot_tenor('6_m')

## Using Pandas (already done)

In [ ]:
df = pd.read_csv('../Plotly-Dashboards-with-Dash/SourceData/nst-est2017-alldata.csv')

In [ ]:
df2 = df[df['DIVISION'] == '1']
df2.set_index('NAME',inplace=True)
df2 = df2[[col for col in df2.columns if col.startswith('POP')]]

In [ ]:
data = [go.Scattergl(x=df2.columns,y=df2.loc[name],
                     mode='lines+markers',name=name) for name in df2.index]

pyo.plot(data)

In [4]:
def tenor_laber_maker(tenor):
    tenor,horizon = tenor.split('_')
    if horizon == 'y':
        label = tenor + ' year yield'
    else:
        label = tenor + ' month yield'
        
    return(label)

# create multiple tenor curves
def rate_trace_maker(tenor,df):
    trace = go.Scattergl(x=df['date'],
                         y=df[tenor],mode='lines',
                         name=tenor_laber_maker(tenor),
                         )
    return(trace)

def plot_tenors(traces):
    
    layout = go.Layout(title='Multiple Rate Plots', 
                       xaxis=dict(title='Date'),yaxis=dict(title='Rate'), 
                       hovermode='closest')
    fig = go.Figure(data=traces,layout=layout)
    pyo.plot(fig)
    

In [5]:
tenors = ['1_m','6_m','10_y','30_y']

In [6]:
traces = []
for tenor in tenors:
    temp = rate_trace_maker(tenor,df)
    traces.append(temp)

In [7]:
plot_tenors(traces)

## Bar Charts

In [ ]:
# get covid19 data
import pandas as pd
import io
import requests
url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
df_sum = df.groupby(by='state').agg(max).reset_index()[['state','cases','deaths']]

In [ ]:
df_sum.sort_values(by=['deaths','cases'],ascending=False,inplace=True)

In [ ]:
trace1 = go.Bar(x=df_sum['state'],y=df_sum['deaths'],
                marker=dict(color='#a60000'),name='Deaths')
trace2 = go.Bar(x=df_sum['state'],y=df_sum['cases'],
                marker=dict(color='#ebab34'),name='Cases')

data = [trace1,trace2]
layout = go.Layout(title='Total CCP Virus Deaths in US States',
                   xaxis=dict(title='State'),yaxis=dict(title='Total Deaths'),
                   barmode='stack')
fig = go.Figure(data=data,layout=layout)
pyo.plot(fig)

## Bubble Charts

In [ ]:
tickers = ['AAPL', 'MSFT', 'NDAQ']
date = { 'gte': '2015-12-31', 'lte': '2020-04-15' }

In [ ]:
import quandl as qd

def get_ticker_data(tickers,date):
    qd.ApiConfig.api_key = 'sy31ewCypsNZeYvWPmx_'

    # get the table for daily stock prices and,
    # filter the table for selected tickers, columns within a time range
    # set paginate to True because Quandl limits tables API to 10,000 rows per call

    data = qd.get_table('WIKI/PRICES', ticker = tickers, 
                            qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                            date = date, 
                            paginate=True)

    data.set_index('date',inplace=True)
    data = data.pivot(columns='ticker')

    data['AAPL_return']=data.adj_close.AAPL.pct_change()
    data['MSFT_return']=data.adj_close.MSFT.pct_change()
    data['NDAQ_return']=data.adj_close.NDAQ.pct_change()
    

    return(data)

In [ ]:
df = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/mpg.csv')

In [ ]:
df['cylinders'] = df['cylinders'].astype('category')

In [ ]:
traces = []
unique_cyls = list(df.cylinders.unique())
for cyl in unique_cyls:
    df_temp = df[df.cylinders==cyl]
    trace = go.Scattergl(y=df_temp['horsepower'],x=df_temp['mpg'],text=df_temp['name'],mode='markers',
                     marker=dict(size=df['weight']/200),name=str(cyl) + ' Cylinders',)
    traces.append(trace)

In [ ]:
layout = go.Layout(title='Bubble Chart',xaxis=dict(title='MPG'),yaxis=dict(title='HP'))
fig = go.Figure(data=traces,layout=layout)
pyo.plot(fig)

## Box Plots

In [ ]:
tickers = ['AAPL', 'MSFT', 'NDAQ']
date = { 'gte': '2015-12-31', 'lte': '2020-04-15' }

In [ ]:
returns = get_ticker_data(tickers,date)

In [ ]:
trace1 = go.Box(y=returns.AAPL_return,boxpoints='all',jitter=0.5,pointpos=0)
trace2 = go.Box(y=returns.NDAQ_return,boxpoints='all',jitter=0.5,pointpos=0)

In [ ]:
pyo.plot([trace1,trace2])

## Histograms

In [ ]:
trace1 = go.Histogram(x=returns.AAPL_return,name='AAPL',xbins=dict(size=0.001))
trace2 = go.Histogram(x=returns.NDAQ_return,name='NDAQ',xbins=dict(size=0.001))
trace3 = go.Histogram(x=returns.MSFT_return,name='MSFT',xbins=dict(size=0.001))

In [ ]:
pyo.plot([trace1,trace2])

In [ ]:
fig = make_subplots(rows=1,cols=3,subplot_titles=['AAPL','NDAQ','MSFT'],
                          shared_yaxes=True)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,1,3)
pyo.plot(fig)

## Distribution Plots

In [ ]:
import plotly.figure_factory as ff

In [ ]:
hdata = [returns.AAPL_return.dropna(),returns.NDAQ_return.dropna(),returns.MSFT_return.dropna()]

In [ ]:
fig = ff.create_distplot(hist_data=hdata,
                         group_labels=['AAPL','NDAQ','MSFT'],bin_size=0.001)
pyo.plot(fig)

## Heatmaps

In [ ]:
df1 = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/2010YumaAZ.csv')
df2 = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/2010SitkaAK.csv')
df3 = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/2010SantaBarbaraCA.csv')

In [ ]:
trace1 = go.Heatmap(x=df1['DAY'],y=df1['LST_TIME'],z=df1['T_HR_AVG'],zmin=4,zmax=40)
trace2 = go.Heatmap(x=df2['DAY'],y=df2['LST_TIME'],z=df2['T_HR_AVG'],zmin=4,zmax=40)
trace3 = go.Heatmap(x=df3['DAY'],y=df3['LST_TIME'],z=df3['T_HR_AVG'],zmin=4,zmax=40)

### Subplots

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
fig = make_subplots(rows=1,cols=3,subplot_titles=['Yuma','Sitka','Santa Barbara'],
                          shared_yaxes=True)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,1,3)
fig['layout'].update(title='Temps')
pyo.plot(fig)

In [ ]:
layout=go.Layout(title='Temps')
fig = go.Figure([trace1],layout=layout)
pyo.plot(fig)

In [ ]:
a